In [1]:
from torch import cuda, nn

import os
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import matplotlib as plt
from matplotlib import pyplot
from torchvision import transforms, utils
from torchvision.datasets import MNIST
import torch.nn.functional as F
import numpy as np
import torch.optim as optim

In [2]:
def flatten(x):
    x.view(6000,1,-1)

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

trainset = MNIST ('./data', train=True, download= True, transform= transform)
testset = MNIST('./data', train=False, download= True, transform=transform)

mnist_train_loader= torch.utils.data.DataLoader(trainset,batch_size=10, shuffle=True, num_workers=2)
mnist_test_loader= torch.utils.data.DataLoader(testset,batch_size=10, shuffle=True, num_workers=2)



In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)




cuda:0


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.h1 = nn.Linear(784,500)
        self.h2 = nn.Linear(500,500)
        self.h3 = nn.Linear(500,10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.sigmoid(self.h1(x))
        x = F.sigmoid(self.h2(x))
        x = F.softmax(self.h3(x))
        return x

    
net = Net()
net.to(device)



Net(
  (h1): Linear(in_features=784, out_features=500, bias=True)
  (h2): Linear(in_features=500, out_features=500, bias=True)
  (h3): Linear(in_features=500, out_features=10, bias=True)
)

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

In [6]:
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(mnist_train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i %2000 == 1999:
            print('[%d,%5d] loss: %.3f'%
                 (epoch + 1, i+1, running_loss/2000))
            running_loss = 0.0
            
print('Finished Training')

C:\Anaconda3\lib\site-packages\torch\nn\functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


[1, 2000] loss: 1.652
[1, 4000] loss: 1.540
[1, 6000] loss: 1.527
[2, 2000] loss: 1.517
[2, 4000] loss: 1.513
[2, 6000] loss: 1.512
[3, 2000] loss: 1.505
[3, 4000] loss: 1.509
[3, 6000] loss: 1.504
[4, 2000] loss: 1.499
[4, 4000] loss: 1.501
[4, 6000] loss: 1.500
[5, 2000] loss: 1.496
[5, 4000] loss: 1.497
[5, 6000] loss: 1.498
[6, 2000] loss: 1.495
[6, 4000] loss: 1.495
[6, 6000] loss: 1.497
[7, 2000] loss: 1.494
[7, 4000] loss: 1.492
[7, 6000] loss: 1.494
[8, 2000] loss: 1.491
[8, 4000] loss: 1.492
[8, 6000] loss: 1.493
[9, 2000] loss: 1.491
[9, 4000] loss: 1.491
[9, 6000] loss: 1.492
[10, 2000] loss: 1.489
[10, 4000] loss: 1.491
[10, 6000] loss: 1.491
Finished Training


In [7]:
classes = ('0', '1', '2', '3', '4', '5', '6', '7','8','9')
for _, data in enumerate(mnist_test_loader):
    images, labels = data[0].to(device), data[1].to(device)
    

outputs = net(images)

_, predicted = torch.max(outputs,1)
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range (4)))


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


Predicted:      0     9     8     9


In [8]:
correct = 0
total = 0
with torch.no_grad():
    for _,data in enumerate(mnist_test_loader):
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


Accuracy of the network on the 10000 test images: 96 %


In [15]:
class CNNet(nn.Module):
    def __init__(self):
        super(CNNet, self).__init__()
        self.conv1 = nn.Conv2d(1,28,4)
        self.conv2 = nn.Conv2d(28,50,5)
        self.pool = nn.MaxPool2d(2,2)
        self.l1 = nn.Linear(800, 23)
        self.l2 = nn.Linear(23,40)
        self.l3 = nn.Linear(40,10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.softmax(self.l3(x))
        return x
cnnet = CNNet()
cnnet.to(device)
    

CNNet(
  (conv1): Conv2d(1, 28, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(28, 50, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (l1): Linear(in_features=800, out_features=23, bias=True)
  (l2): Linear(in_features=23, out_features=40, bias=True)
  (l3): Linear(in_features=40, out_features=10, bias=True)
)

In [16]:
cnnetoptim = optim.Adam(cnnet.parameters(), lr=1e-3)
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(mnist_train_loader):
        images, labels = data[0].to(device), data[1].to(device)
        cnnetoptim.zero_grad()
        outputs = cnnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        cnnetoptim.step()
        running_loss += loss.item()
        if i %2000 == 1999:
            print('[%d,%5d] loss: %.3f'%
                 (epoch + 1, i+1, running_loss/2000))
            running_loss = 0.0

print("Finished Training")

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[1, 2000] loss: 1.585
[1, 4000] loss: 1.508
[1, 6000] loss: 1.498
[2, 2000] loss: 1.493
[2, 4000] loss: 1.493
[2, 6000] loss: 1.494
[3, 2000] loss: 1.490
[3, 4000] loss: 1.489
[3, 6000] loss: 1.489
[4, 2000] loss: 1.491
[4, 4000] loss: 1.489
[4, 6000] loss: 1.495
[5, 2000] loss: 1.491
[5, 4000] loss: 1.487
[5, 6000] loss: 1.489
[6, 2000] loss: 1.488
[6, 4000] loss: 1.485
[6, 6000] loss: 1.492
[7, 2000] loss: 1.487
[7, 4000] loss: 1.489
[7, 6000] loss: 1.489
[8, 2000] loss: 1.488
[8, 4000] loss: 1.485
[8, 6000] loss: 1.494
[9, 2000] loss: 1.488
[9, 4000] loss: 1.491
[9, 6000] loss: 1.498
[10, 2000] loss: 1.492
[10, 4000] loss: 1.493
[10, 6000] loss: 1.492
Finished Training


In [17]:
correct = 0
total = 0
with torch.no_grad():
    for _,data in enumerate(mnist_test_loader):
        images, labels = data[0].to(device), data[1].to(device)
        outputs = cnnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 10000 test images: 97 %


In [18]:
for _, data in enumerate(mnist_test_loader):
    images, labels = data[0].to(device), data[1].to(device)
    

outputs = cnnet(images)

_, predicted = torch.max(outputs,1)
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range (4)))


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Predicted:      7     0     7     0
